### `MaxPool2D` code

In [1]:
import numpy as np
import torch # needed for tests
from tqdm import tqdm # needed for tests

In [2]:
class MaxPool2D:
    ''' Computes maxpool given the input parameters '''
    
    ''' * The class implementation will be along the lines of torch.nn.MaxPool2d in order to 
          enable comparison of this NumPy only implementation and seamless testing
    '''
    '''
        TODO:
        * Replace `torch.round()` with `np.allclose()` for tests
        * Optimizing code
    '''
    
    def __init__(
        self, 
        kernel_size, 
        padding = (0, 0), 
        stride = (1, 1), 
        dilation = (1, 1), 
        verbose = False
        ):
        super(MaxPool2D, self).__init__()
        
        ''' mandatory parameters '''
        if isinstance(kernel_size, tuple):
            self.kernel_size = kernel_size
        elif isinstance(kernel_size, int):
            self.kernel_size = (kernel_size, kernel_size)
        else:
            raise Exception('invalid input parameters: kernel_size should either be an int or a tuple')
        
        ''' optional parameters '''
        if isinstance(padding, str):
            if padding == 'valid':
                self.padding = (0, 0)
            elif padding == 'same':
                raise Exception("invalid input parameters: padding = 'same' not yet supported")
            else:
                raise Exception('invalid input parameters: padding is not valid')
        elif isinstance(padding, tuple):
            if padding[0] >= 0 and padding[1] >= 0:
                self.padding = padding
            else:
                raise Exception('invalid input parameters: padding is not valid')
        elif isinstance(padding, int):
            if padding >= 0:
                self.padding = (padding, padding)
            else:
                raise Exception('invalid input parameters: padding is not valid')
        else:
            raise Exception('invalid input parametersL padding should be either an int or a tuple')
        if isinstance(stride, tuple):
            if stride[0] >= 1 and stride[1] >= 1:
                self.stride = stride
            else:
                raise Exception('invalid input parameters: stride is not valid')
        elif isinstance(stride, int):
            if stride >= 1:
                self.stride = (stride, stride)
            else:
                raise Exception('invalid input parameters: stride is not valid')
        else:
            raise Exception('invalid input parameters: stride should be either an int or a tuple')
        if isinstance(dilation, tuple):
            if dilation[0] >= 1 and dilation[1] >= 1:
                self.dilation = dilation
            else:
                raise Exception('invalid input parameters: dilation is not valid')
        elif isinstance(dilation, int):
            if dilation >= 1:
                self.dilation = (dilation, dilation)
            else:
                raise Exception('invalid input parameters: dilation is not valid')
        else:
            raise Exception('invalid input parameters: dilation should be either an int or a tuple')
        
        ''' optional parameters (dummy, yet to be implemented)'''
        
        
        ''' additional parameters (different from torch.nn.Conv2D)'''
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        self.verboseprint('*** parameters ***')
        self.verboseprint('kernel_size: {}'.format(self.kernel_size))
        self.verboseprint('padding: {}, stride: {}, dilation factor: {}'.format(self.padding, self.stride, self.dilation))
        self.verboseprint('\n')
    
    def forward(self, _input):
        ''' forward pass to perform convolution '''
        
        ''' do error checking '''
        _input_n, _input_c, _input_h, _input_w = _input.shape
        if _input_h + 2 * self.padding[0] < self.dilation[0] * (self.kernel_size[0] - 1) + 1: # check if (dilated) ker_h is valid
            raise Exception('invalid input parameters: kernel height is larger than input height')
        if _input_w + 2 * self.padding[1] < self.dilation[1] * (self.kernel_size[1] - 1) + 1: # check if (dilated) ker_w is valid
            raise Exception('invalid input parameters: kernel width is larger than input width')
        if ((_input_h + 2 * self.padding[0] - (self.dilation[0] * (self.kernel_size[0] - 1) + 1)) / self.stride[0]) + 1 < 0: # check if out_h is valid
            raise Exception('invalid input parameters: output height is negative')
        if ((_input_w + 2 * self.padding[1] - (self.dilation[1] * (self.kernel_size[1] - 1) + 1)) / self.stride[1]) + 1 < 0: # check if out_w is valid
            raise Exception('invalid input parameters: output width is negative')
        if self.padding[0] > self.kernel_size[0] // 2: # as PyTorch mandates this
            raise Exception('invalid input parameters: padding height is larger than half of kernel height')
        if self.padding[1] > self.kernel_size[1] // 2: # as PyTorch mandates this
            raise Exception('invalid input parameters: padding width is larger than half of kernel width')
        
        ''' add zero padding based on the input parameters '''
        if self.padding != (0, 0):
            _input = np.array([[np.pad(channel, ((self.padding[0], self.padding[0]), (self.padding[1], self.padding[1])), 'constant', constant_values = -np.inf) for channel in batch] for batch in _input])    
            self.verboseprint('*** padded input image ***')
            self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(_input.shape[0], _input.shape[1], _input.shape[2], _input.shape[3]))
            self.verboseprint(_input)
            self.verboseprint('\n')
        
        ''' dilate a kernel '''
        dil_ker_h = self.dilation[0] * (self.kernel_size[0] - 1) + 1
        dil_ker_w = self.dilation[1] * (self.kernel_size[1] - 1) + 1
        self.kernel_size = (dil_ker_h, dil_ker_w)
        self.verboseprint('*** dilated kernels ***')
        self.verboseprint('dilation factor: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.dilation, int(_input_c), self.kernel_size[0], self.kernel_size[1]))
        self.verboseprint('\n')
        
        ''' compute output volume from the input and kernel parameters '''
        _input_n, _input_c, _input_h, _input_w = _input.shape
        out_n = int(_input_n)
        out_c = int(_input_c)
        out_h = int((_input_h - self.kernel_size[0]) / self.stride[0]) + 1
        out_w = int((_input_w - self.kernel_size[1]) / self.stride[1]) + 1
        output = np.zeros([out_n, out_c, out_h, out_w])
        
        ''' parse through every element of the output and compute the convolution value for that element '''
        for b in range(out_n):
            for k in range(out_c):
                for h in range(out_h):
                    for w in range(out_w):
                        # convolve kernel over the input slices
                        self.verboseprint('kernel indices, image indices')
                        self.verboseprint('[n, c, h, w]', '[n, c, h, w]')
                        convol_sum = -np.inf
                        ker_h = self.kernel_size[0]
                        ker_w = self.kernel_size[1]
                        for h_ker in range(ker_h):
                            for w_ker in range(ker_w):
                                self.verboseprint([b, k, h_ker + self.stride[0] * h, w_ker + self.stride[1] * w])
                                if _input[b][k][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w] > convol_sum: # works for all 1 <= `groups` <= in_channels
                                    convol_sum = _input[b][k][h_ker + self.stride[0] * h][w_ker + self.stride[1] * w]
                        self.verboseprint('\n')
                        output[b, k, h, w] += convol_sum
        self.verboseprint('*** MaxPool2D output ***')
        output_shape = output.shape
        self.verboseprint('output batches: {}, ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2], output_shape[3]))
        assert((out_n, out_c, out_h, out_w) == output_shape)
        self.verboseprint(output)
        self.verboseprint('\n')
        return output

### Standalone test (random kernel, random input)

In [3]:
in_channels = 2 # input channels
kernel_size = (2, 2) # kernel size

padding = (0, 0) # padding (optional)
stride = (2, 2) # stride (optional)
dilation = (1, 1) # dilation factor (optional)

in_batches = 1 # input batches
in_h = 4 # input height
in_w = 4 # input weight

_input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters

In [4]:
# get MaxPool2D output with the random inputs

maxpool2d = MaxPool2D(kernel_size, stride = stride, padding = padding, dilation = dilation) # call an instance of the class with the input parameters 
_output = maxpool2d.forward(_input) # perform convolution
print("*** MaxPool2D output ***")
print(_output)

*** MaxPool2D output ***
[[[[0.97531315 0.97531315 0.98736302 0.98736302 0.9849193  0.9991104
    0.98521671 0.97823002 0.97823002]
   [0.97531315 0.97531315 0.98736302 0.98736302 0.98608482 0.9991104
    0.98521671 0.97823002 0.97823002]
   [0.9839269  0.97531315 0.98736302 0.98736302 0.98608482 0.9991104
    0.98521671 0.99842113 0.99842113]
   [0.9839269  0.98016109 0.98736302 0.98736302 0.98608482 0.98608482
    0.98041436 0.99842113 0.99842113]
   [0.9839269  0.98016109 0.98016109 0.95893345 0.95893345 0.9186637
    0.9560461  0.99842113 0.99842113]
   [0.99113498 0.99113498 0.98016109 0.95893345 0.95893345 0.93390951
    0.9560461  0.99842113 0.99842113]
   [0.99113498 0.99657488 0.99657488 0.99094851 0.99094851 0.93390951
    0.93366799 0.93366799 0.92043122]
   [0.99113498 0.99941284 0.99657488 0.99094851 0.99094851 0.93390951
    0.93366799 0.93366799 0.9325343 ]
   [0.99113498 0.99941284 0.99657488 0.99094851 0.99094851 0.93390951
    0.93366799 0.93366799 0.9325343 ]]

  [[0

In [5]:
# get PyTorch output with the same random inputs as above

x = torch.DoubleTensor(_input)
m = torch.nn.MaxPool2d(kernel_size, stride = stride, padding = padding, dilation = dilation)
output = m(x)
print("*** PyTorch output ***")
print(output)

*** PyTorch output ***
tensor([[[[0.9753, 0.8690, 0.9874, 0.9363, 0.9849, 0.9852, 0.9705, 0.9521,
           0.7587],
          [0.9615, 0.9601, 0.9119, 0.9259, 0.9861, 0.9804, 0.9236, 0.9782,
           0.9560],
          [0.9753, 0.8690, 0.9874, 0.9363, 0.9297, 0.9852, 0.8018, 0.9521,
           0.9984],
          [0.8824, 0.9802, 0.9446, 0.9589, 0.9861, 0.9804, 0.9560, 0.9782,
           0.9669],
          [0.9382, 0.8244, 0.8834, 0.7669, 0.9297, 0.8841, 0.7718, 0.9146,
           0.9984],
          [0.8118, 0.9911, 0.9446, 0.9589, 0.8360, 0.9339, 0.9560, 0.8849,
           0.9669],
          [0.9382, 0.9966, 0.8064, 0.9284, 0.9909, 0.8841, 0.9337, 0.8839,
           0.9111],
          [0.9831, 0.9911, 0.9108, 0.8175, 0.7419, 0.9339, 0.8838, 0.9173,
           0.8331],
          [0.6918, 0.9966, 0.7075, 0.9578, 0.9909, 0.7370, 0.9337, 0.8839,
           0.8485]],

         [[0.7960, 0.9901, 0.9655, 0.5994, 0.6637, 0.8431, 0.9523, 0.9630,
           0.9311],
          [0.8734, 0.9454

In [6]:
# compare outputs of Conv2D and PyTorch
print(torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output))) # need to round the output due to precision difference

False


### Extensive tests (random kernel, random input)

In [7]:
def valid_params(num_tests):
    ''' generates `num_tests` number of valid input and kernel parameters '''
    np.random.seed(1729)
    params_list = []
    sample_count = 0
    while sample_count < num_tests:
        in_channels = np.random.randint(20) + 1 # input channels
        
        kernel_h = np.random.randint(20) + 1
        kernel_w = np.random.randint(20) + 1
        kernel_size = (kernel_h, kernel_w) # kernel size
        
        padding_h = np.random.randint(10) + 1
        padding_w = np.random.randint(10) + 1
        padding = (padding_h, padding_w) # padding (optional)
        stride_h = np.random.randint(5) + 1
        stride_w = np.random.randint(5) + 1
        stride = (stride_h, stride_w) # stride (optional)
        dilation_h = np.random.randint(10) + 1
        dilation_w = np.random.randint(10) + 1
        dilation = (dilation_h, dilation_w) # dilation factor (optional)
        
        in_batches = np.random.randint(5) + 1 # input batches
        in_h = np.random.randint(30) + 5 # input height
        in_w = np.random.randint(30) + 5 # input weight
    
        ker_h_flag, ker_w_flag, out_h_flag, out_w_flag, pad_h_flag, pad_w_flag = True, True, True, True, True, True
        
        if in_h + 2 * padding_h < dilation_h * (kernel_h - 1) + 1: # check if (dilated) ker_h is valid
            ker_h_flag = False
        if in_w + 2 * padding_w < dilation_w * (kernel_w - 1) + 1: # check if (dilated) ker_w is valid
            ker_w_flag = False
        if ((in_h + 2 * padding_h - (dilation_h * (kernel_h - 1) + 1)) / stride_h) + 1 < 0: # check if out_h is valid
            out_h_flag = False
        if ((in_w + 2 * padding_w - (dilation_w * (kernel_w - 1) + 1)) / stride_w) + 1 < 0: # check if out_w is valid
            out_w_flag = False
        if padding_h > kernel_h // 2: # as PyTorch mandates this
            pad_h_flag = False
        if padding_w > kernel_w // 2: # as PyTorch mandates this
            pad_w_flag = False
    
        if ker_h_flag and ker_w_flag and out_h_flag and out_w_flag and pad_h_flag and pad_w_flag:
            params_list.append({'in_channels': in_channels, 'kernel_size': kernel_size,
                          'padding': padding, 'stride': stride, 'dilation': dilation, 'in_batches': in_batches,
                          'in_h': in_h, 'in_w': in_w})
            sample_count += 1
    return params_list

In [8]:
def run_tests(num_tests):
    ''' sweep different input parameters and test by comparing outputs of Conv2D and PyTorch '''
    
    num_passed = 0
    params_list = valid_params(num_tests)
    print('Number of tests: {}\n\n'.format(len(params_list)))

    for i, params in enumerate(tqdm(params_list)):
        print('Test: {}\nParams: {}'.format(i, params))
        in_channels = params['in_channels'] # input channels
        # out_channels = params['out_channels'] # output channels
        kernel_size = params['kernel_size'] # kernel size

        padding = params['padding'] # padding (optional)
        stride = params['stride'] # stride (optional)
        dilation = params['dilation'] # dilation factor (optional)
        # groups = params['groups'] # groups (optional)

        in_batches = params['in_batches'] # input batches
        in_h = params['in_h'] # input height
        in_w = params['in_w'] # input weight

        _input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
        # kernels = []
        # for k in range(out_channels):
            # kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
            # kernels.append(kernel)

        try:
            # get MaxPool2D output with the random inputs
            maxpool2d = MaxPool2D(kernel_size, stride = stride, padding = padding, dilation = dilation) # call an instance of the class with the input parameters 
            _output = maxpool2d.forward(_input) # perform convolution

            # get PyTorch output with the same random inputs as above
            x = torch.DoubleTensor(_input)
            m = torch.nn.MaxPool2d(kernel_size, stride = stride, padding = padding, dilation = dilation)
            output = m(x)

        except Exception as e:
            print(e)
            print('Result: False\n\n') # treating exception as a failed test
            continue

        # compare outputs of MaxPool2D and PyTorch
        result = torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output)) # need to round the output due to precision difference
        print('Result: {}\n\n'.format(result))
        if result:
            num_passed += 1

    print('{} out of {} ({}%) tests passed'.format(num_passed, num_tests, float(100 * num_passed / num_tests)))


In [9]:
num_tests = 100
run_tests(num_tests)

Number of tests: 100




  2%|▊                                          | 2/100 [00:00<00:07, 12.78it/s]

Test: 0
Params: {'in_channels': 10, 'kernel_size': (16, 20), 'padding': (7, 4), 'stride': (4, 5), 'dilation': (1, 1), 'in_batches': 2, 'in_h': 15, 'in_w': 26}
Result: True


Test: 1
Params: {'in_channels': 4, 'kernel_size': (20, 12), 'padding': (10, 1), 'stride': (1, 3), 'dilation': (2, 2), 'in_batches': 2, 'in_h': 33, 'in_w': 23}
Result: True


Test: 2
Params: {'in_channels': 2, 'kernel_size': (18, 7), 'padding': (5, 2), 'stride': (5, 1), 'dilation': (2, 4), 'in_batches': 5, 'in_h': 29, 'in_w': 25}
Result: True


Test: 3
Params: {'in_channels': 7, 'kernel_size': (10, 13), 'padding': (2, 1), 'stride': (4, 3), 'dilation': (3, 2), 'in_batches': 5, 'in_h': 30, 'in_w': 25}


  5%|██▏                                        | 5/100 [00:00<00:05, 18.64it/s]

Result: True


Test: 4
Params: {'in_channels': 3, 'kernel_size': (11, 5), 'padding': (1, 2), 'stride': (1, 5), 'dilation': (2, 8), 'in_batches': 1, 'in_h': 33, 'in_w': 34}
Result: True


Test: 5
Params: {'in_channels': 20, 'kernel_size': (6, 14), 'padding': (1, 5), 'stride': (1, 1), 'dilation': (2, 1), 'in_batches': 1, 'in_h': 10, 'in_w': 16}
Result: True


Test: 6
Params: {'in_channels': 20, 'kernel_size': (15, 11), 'padding': (7, 3), 'stride': (3, 3), 'dilation': (2, 3), 'in_batches': 4, 'in_h': 28, 'in_w': 34}


...

Result: True


Test: 95
Params: {'in_channels': 14, 'kernel_size': (6, 20), 'padding': (2, 2), 'stride': (4, 2), 'dilation': (1, 1), 'in_batches': 1, 'in_h': 12, 'in_w': 26}
Result: True


Test: 96
Params: {'in_channels': 10, 'kernel_size': (19, 7), 'padding': (1, 2), 'stride': (4, 1), 'dilation': (1, 4), 'in_batches': 1, 'in_h': 33, 'in_w': 24}
Result: True


Test: 97
Params: {'in_channels': 4, 'kernel_size': (19, 6), 'padding': (5, 2), 'stride': (4, 1), 'dilation': (1, 3), 'in_batches': 5, 'in_h': 33, 'in_w': 33}


 99%|█████████████████████████████████████████▌| 99/100 [00:28<00:00,  3.14it/s]

Result: True


Test: 98
Params: {'in_channels': 4, 'kernel_size': (10, 9), 'padding': (5, 1), 'stride': (4, 4), 'dilation': (1, 1), 'in_batches': 4, 'in_h': 13, 'in_w': 23}
Result: True


Test: 99
Params: {'in_channels': 12, 'kernel_size': (16, 17), 'padding': (5, 6), 'stride': (1, 2), 'dilation': (2, 1), 'in_batches': 5, 'in_h': 30, 'in_w': 5}


100%|█████████████████████████████████████████| 100/100 [00:28<00:00,  3.46it/s]

Result: True


95 out of 100 (95.0%) tests passed
